In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable


import argparse, os, sys, subprocess
import numpy as np
from tqdm import tqdm
from glob import glob
from os.path import *

import models, losses
from utils import flow_utils, tools
from networks import FlowNetS
from utils import tools
from path import Path
from glob import glob as glob
from natsort import natsorted
from imageio import imread, imwrite

In [1]:
import torch
model = torch.load('/user_data/vayzenbe/GitHub_Repos/LiMA/Weights/flownet2S.pt')

In [3]:
dir(model.__module__)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',


In [2]:
img_pairs = []
ext = 'jpg'
data_dir = Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge')
test_files = data_dir.files('*1.{}'.format(ext))
for file in test_files:
    img_pair = file.parent / (file.stem[:-1] + '2.{}'.format(ext))
    if img_pair.isfile():
        img_pairs.append([file, img_pair])

NameError: name 'Path' is not defined

In [59]:
img_pairs

[[Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_21.jpg'),
  Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_22.jpg')],
 [Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_211.jpg'),
  Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_212.jpg')],
 [Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_101.jpg'),
  Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_102.jpg')],
 [Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_141.jpg'),
  Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_142.jpg')],
 [Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_61.jpg'),
  Path('/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_62.jpg')],
 [Path('/user_data/vayzenbe/GitHub_Repo

In [55]:
imfiles = natsorted(glob(f'{data_dir}/*.jpg'))

In [62]:
img_pairs = []
for fn in range(0,len(imfiles)-1):
    img_pairs.append([imfiles[fn], imfiles[fn+1]])
    

In [63]:
img_pairs

[['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_1.jpg',
  '/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_2.jpg'],
 ['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_2.jpg',
  '/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_3.jpg'],
 ['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_3.jpg',
  '/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_4.jpg'],
 ['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_4.jpg',
  '/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_5.jpg'],
 ['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_5.jpg',
  '/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_6.jpg'],
 ['/user_data/vayzenbe/GitHub_Repos/LiMA/Frames/Figure_23_Bulge/Figure_23_Bulge_6.jpg',
  '/user_data/vayzenbe/GitH

In [3]:
model = torch.load('weights/flownet2S.pt')
checkpoint = torch.load('weights/FlowNet2-S_checkpoint.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
encoder = nn.Sequential(*list(model.children())[:-1])

In [4]:
model = torch.load('weights/flownet2S.pt')

In [11]:
checkpoint['state_dict']

OrderedDict([('conv1.0.weight',
              tensor([[[[-1.7613e-01,  4.4566e-02,  4.5786e-02,  ...,  2.2862e-02,
                          9.8137e-02,  7.2715e-02],
                        [-8.0132e-02,  1.7608e-02, -4.8919e-03,  ..., -7.4431e-02,
                         -8.0240e-02,  1.5525e-01],
                        [-6.2233e-02, -1.0890e-01, -1.0670e-01,  ..., -1.0271e-01,
                          3.2037e-03,  6.5534e-02],
                        ...,
                        [-6.3074e-02,  4.3976e-03, -2.5781e-02,  ...,  2.1247e-02,
                          8.8751e-02,  5.2259e-02],
                        [-1.0149e-01, -7.5587e-02,  5.6959e-02,  ..., -1.0657e-02,
                         -5.4725e-02,  7.0781e-02],
                        [-1.0983e-01, -1.7767e-02, -4.2275e-02,  ...,  4.1326e-02,
                          1.6339e-02,  7.1811e-02]],
              
                       [[ 2.0861e-01,  1.8536e-02,  5.9876e-02,  ..., -6.0212e-02,
                          8.11

In [15]:
model.module.model.load_state_dict(checkpoint['state_dict']).load_state_dict(model, checkpoint['state_dict'])

AttributeError: type object 'FlowNet2S' has no attribute 'module'

In [17]:
model.modules.load_state_dict(checkpoint['state_dict']).load_state_dict(model, checkpoint['state_dict'])

AttributeError: 'function' object has no attribute 'load_state_dict'

In [19]:
print(model.children())

TypeError: children() missing 1 required positional argument: 'self'